In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv


## Read Input Files

In [19]:
train_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa', 'train_features.csv'))
test_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa' , 'test_features.csv'))

train_target = pd.read_csv(os.path.join('/kaggle/input/lish-moa','train_targets_scored.csv'))

## Print Shape of Data

In [164]:
print("train_data.shap: {}  , train_target.shap: {}  , test_data.shap: {} ".format(train_data.shape , train_target.shape , test_data.shape))

train_data.shap: (23814, 876)  , train_target.shap: (23814, 207)  , test_data.shap: (3982, 876) 


In [165]:
NGTypeFeature = sum(train_data.columns.to_series().str.contains('g-') == True )
NCTypeFeature = sum(train_data.columns.to_series().str.contains('c-') == True )

print("NGTypeFeature = {} \nNCTypeFeature = {}".format(NGTypeFeature , NCTypeFeature))

NGTypeFeature = 772 
NCTypeFeature = 100


## Define a Neural Network To Train

In [166]:
import torch.nn.functional as F
import torch.nn as nn

class MultiLabelClassifer(nn.Module):
    #TO-DO: Documentation
    def __init__(self , input_feature , hidden_dim , output_dim):
        super(MultiLabelClassifer , self).__init__()
        
        self.fc1 = nn.Linear(in_features = input_feature , out_features = 2000)
        #TO-DO: As close features are related here, convoluation layer will help here
        #TO-DO: Later add and play with Convolutatin layer, to see accurracy difference
        self.fc2 = nn.Linear( in_features = 2000 , out_features  = 4000)
        self.fc3 = nn.Linear( in_features = 4000 , out_features  = 1000)
        self.fc4 = nn.Linear(in_features = 1000 , out_features = 400)
        self.fc5 = nn.Linear(in_features = 400 , out_features = output_dim)
        self.drop = nn.Dropout(0.4) # dropout with 30% prob
        
        self.sig = nn.Sigmoid()
        #TO-DO: Define output layer so that it be as per Question requirement
    def forward(self, x):
        
        # (1) input layer
        
        t = x
        
        
        # (2) Hidden Linear Layer
        
        t = self.fc1(t)
        t = F.relu(t)
        #3
        
        t = self.fc2(t)
        t = F.relu(t)
        #4
        
        t= self.fc3(t)
        t = F.relu(t)
        t = self.drop(t)
        
        #5
        
        t = self.fc4(t)
        t = F.relu(t)
        #6
        
        t = self.drop(t)
        
        t = self.fc5(t)
        t = F.relu(t)
        
        # (7) Output Layer
        
        t = self.sig(t)
        
        return t
        

In [167]:
train_rf = train_data.copy()
train_data.head()

test_rf = test_data.copy()

## One hot coding of input feature and Normalization

In [168]:
import torch.utils.data
import torch
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,Normalizer

ignore_columns = ['sig_id','cp_type']

train_columns = [x for x in train_data.columns if x not in ignore_columns]
train = train_rf[train_columns]
#print(train.head())
test = test_rf[train_columns]
target = train_target.iloc[:,1:].values

transform = ColumnTransformer([('o',OneHotEncoder(),[0,1]),('s',Normalizer(),list(range(3,train.shape[1])))])
train = transform.fit_transform(train)
test = transform.transform(test)


In [31]:
#train_rf['cp_dose'] = train_data['cp_dose'].replace({"D1":1 , "D2":2})
#train_rf['cp_type'] = train_rf['cp_type'].replace({"trt_cp":1 , "ctl_vehicle":2})

#test_rf['cp_dose'] = test_data['cp_dose'].replace({"D1":1 , "D2":2})
#test_rf['cp_type'] = test_data['cp_type'].replace({"trt_cp":1 , "ctl_vehicle":2})

In [177]:
print(train[0])

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  0.00000000e+00  2.50303421e-02 -1.11260925e-02 -2.78623567e-02
 -8.72493903e-03 -4.54199501e-02 -4.58687638e-02 -1.46313278e-03
  2.49001861e-02 -4.13357451e-03  5.30946650e-02  6.86685016e-03
  2.50168776e-02 -1.80198715e-02  8.02927774e-03 -2.92985607e-02
 -3.57659666e-02  2.84637671e-02  7.97990823e-03 -1.65791794e-02
 -2.55285253e-02 -5.09852404e-02 -5.33190718e-02  3.11476733e-02
  1.97163874e-02  1.19563979e-02  8.55887795e-03  7.30668762e-03
 -1.28046559e-02  2.61164713e-02  1.31681950e-02 -2.50617590e-02
 -4.11113382e-03 -1.35092935e-02 -6.89826712e-03  9.86492592e-03
  1.33073273e-02 -2.26875344e-02 -2.29747751e-02 -9.70335297e-03
 -1.55738366e-03 -1.15165605e-02 -5.37678856e-02  1.39850360e-02
 -4.63624589e-02  9.90980729e-03 -5.12994100e-03 -1.12652248e-03
  1.57084808e-03  1.89982854e-02 -5.48001572e-03 -2.21130528e-02
  2.74629125e-02  2.11615677e-02 -1.57533621e-03 -7.20346046e-03
 -3.02949272e-02  1.01566

## Get Data Tensor

In [178]:
print(type(train))
train_data_tensor =  torch.from_numpy(train.astype(np.float32))
train_target_tensor = torch.from_numpy(target.astype(np.float32))

<class 'numpy.ndarray'>


In [184]:
print("max_val {}, min_val {}",train_data_tensor.max() , train_data_tensor.min() )
#Value range b/w -1 to 1 implies that it has been normalized

max_val {}, min_val {} tensor(1.) tensor(-0.4424)


In [185]:
print(train_target_tensor)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [186]:
print(sum(sum(train_target_tensor)))

tensor(16844.)


In [187]:
print("type of train_data_tensor: {}",type(train_data_tensor) )
train_set = torch.utils.data.TensorDataset(train_data_tensor , train_target_tensor)
print(train_set)

type of train_data_tensor: {} <class 'torch.Tensor'>


## Get dataloader

In [189]:
batch_size = 300
train_loader = torch.utils.data.DataLoader(train_set , batch_size = batch_size)
print(train_loader)

## Now take Instance of Network

In [198]:
model = MultiLabelClassifer(876 , 1024  , 206 )
#(self , input_feature , hidden_dim , output_dim):
#model = ModelTWO(876 , 206 , 1024)

# Analyser Sample Formed Data

In [199]:
batch = next(iter(train_loader))
feature, label = batch

In [200]:
print(batch[0].shape)
print(batch[0][0].shape) #THis is one extracted Feature out of 100

torch.Size([300, 876])
torch.Size([876])


In [201]:
print(feature)

tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  9.6001e-03,
          1.7059e-02,  1.8742e-02],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  5.6319e-03,
          2.7579e-02,  3.3451e-02],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -7.6819e-03,
         -4.9456e-02,  2.4345e-02],
        ...,
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.1164e-03,
          3.6492e-02,  2.0576e-02],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  8.3413e-04,
         -2.9741e-02,  5.5212e-03],
        [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -5.4815e-02,
         -2.0204e-02, -3.4210e-02]])


In [202]:
print(sum(label[0]))
print(label[0])   #label for 1st feature data

tensor(1.)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


# Now train the model

In [210]:

import torch.optim as optim
learning_rate = [ 1 , .01 , .001 , .0001]

for lr in learning_rate:
    optimizer = optim.Adam(model.parameters() , lr = lr)
    criterion = torch.nn.BCELoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            features , labels  = batch
            optimizer.zero_grad()

            preds = model(features)

            #print(sum(preds>1))
            loss = criterion(preds , labels)


            loss.backward()
            optimizer.step() #update weights

            total_loss += loss.data.item()
            #print("total_loss: {}".format(total_loss))
            #total_correct  += get_num_correct(preds,labels)
        print(" epoch: {} , learning_rate:{} , total_loss: {}".format(epoch, lr, total_loss))

learning_rate:1 , epoch: 0 total_loss: 1139.9748458862305


KeyboardInterrupt: 